In [1]:
import JLD2
import CSV

using Discretizers
using DataFrames

  likely near In[1]:4
  likely near In[1]:4


In [2]:
include("src\\NGSIM.jl")

chmm_em! (generic function with 1 method)

In [4]:
traj_file_num = 1
traj_file = Base.Filesystem.basename(NGSIM.TRAJDATA_PATHS[traj_file_num])
traj_file = traj_file[1:search(traj_file, '.')-1]

DATA_PATH = joinpath("../data", traj_file)

"../data/trajdata_i101_trajectories-0750am-0805am"

In [5]:
!isdir(DATA_PATH) && Base.Filesystem.mkdir(DATA_PATH)

false

In [6]:
# td = load_trajdata(traj_file_num);
JLD2.@load "$(DATA_PATH)/td.jld" td
rd = NGSIM.ROADWAY_101
(TS, id_lookup) = td_sparse(td)
;

# X

In [8]:
fs = [
    ("x", s::VehicleState -> s.posG.x),
    ("y", s::VehicleState -> s.posG.y),
    ("heading", s::VehicleState -> s.posF.ϕ),
    ("velocity", s::VehicleState -> s.v),
    ("lane", s::VehicleState -> length(rd[s.posF.roadind.tag.segment].lanes) - s.posF.roadind.tag.lane + 1),
]

X = Array{Float64}(nnz(TS), length(fs))
@inbounds for (t, s) in TS.nzval |> enumerate
    for (i, (_, f)) in enumerate(fs)
        X[t, i] = f(s)
    end
end

In [9]:
# make a dataframe from the extracted data, and then give them names
df = DataFrame([X TS.rowval])
names!(df, vcat(map(Symbol ∘ first, fs), :frame))

# make frame and lane Ints
for s in [:frame, :lane]
    df[s] = round.(Int, df[s])
end

In [11]:
# estimate acceleration
Δt = td.timestep
V = df[:velocity]
A = zeros(V)
num_traj = TS.n

for i in 1:num_traj
    r = get_trajectory_range(TS.colptr, i)
    T = length(r)
    Vt = view(V, r)
    At = view(A, r)

    # forward diff, Δₕ x
    At[1] = (Vt[2] - Vt[1]) / Δt 
    # central diff, δₕ x
    At[2:(T-1)] = (Vt[3:T] .- Vt[1:(T-2)]) ./ (2*Δt)
    # backward diff, ∇ₕ
    At[T] = (Vt[T] - Vt[T-1]) / Δt
end

df[:acceleration] = A

In [25]:
# the car id
car = zeros(Int, length(V))
for i in 1:num_traj
    r = get_trajectory_range(TS.colptr, i)
    car[r] = i
end

In [28]:
CSV.write(joinpath(DATA_PATH, "X.csv"), df, header=true)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd", IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "../data/trajdata_i101_trajectories-0750am-0805am/X.csv", 65, true, String["x", "y", "heading", "velocity", "lane", "frame", "acceleration", "car"], false)

In [51]:
writecsv(joinpath(DATA_PATH, "traj_lengths.csv"), diff(TS.colptr))

# Pairs

In [ ]:
pair_traj = find_pairs(TS)
pairs = DataFrame([Int, Int, Int, Int, Int], [:car1, :car2, :start, :end, :length], length(pair_traj))

for (e, ((c1, c2), (start_frame, end_frame))) in enumerate(pair_traj)
    pairs[e, :car1] = c1
    pairs[e, :car2] = c2
    pairs[e, :start] = start_frame
    pairs[e, :end] = end_frame
    pairs[e, :length] = end_frame - start_frame + 1
end

In [ ]:
CSV.write(joinpath(DATA_PATH, "pairs.csv"), pairs, header=true)

# Discretize

In [26]:
df_disc = DataFrame(Int, nrow(df), 4)
names!(df_disc, [:velocity, :heading, :lane, :frame])
;

In [27]:
disc1 = LinearDiscretizer(binedges(DiscretizeUniformWidth(150), df[:velocity]))
disc2 = LinearDiscretizer(binedges(DiscretizeUniformWidth(30),df[:heading]))

df_disc[:velocity] = encode(disc1, df[:velocity])
df_disc[:heading] = encode(disc1, df[:heading])
df_disc[:lane] = df[:lane]
df_disc[:frame] = df[:frame]
;

In [28]:
CSV.write(joinpath(DATA_PATH, "X_disc.csv"), df_disc, header=true)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd", IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "../data/trajdata_i101_trajectories-0750am-0805am/X_disc.csv", 36, true, String["velocity", "heading", "lane", "frame"], false)

In [35]:
writecsv(joinpath(DATA_PATH, "X1_bins.csv"), disc1.binedges)

In [34]:
writecsv(joinpath(DATA_PATH, "X2_bins.csv"), disc2.binedges)